In [115]:
#!/usr/bin/env python
#Config
from pyspark import SparkConf, SparkContext, HiveContext
#import re
import numpy as np
import pandas as pd
#import datetime
#from pyspark.mllib.regression import LabeledPoint
#from pyspark.mllib.feature import HashingTF
#from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
#import scipy.sparse as sps
#from pyspark.mllib.linalg import Vectors
#import sklearn
import itertools
import datetime
import sys

try:
    sc.stop()
except NameError:
    pass

conf = SparkConf().set("spark.executor.instances", 8).set("spark.driver.maxResultSize", "4g")
sc = SparkContext()
hc = HiveContext(sc)
sc.setCheckpointDir('checkpoint/')

In [ ]:
section_dict = hc.sql('select * from user_kposminin.yaca_section_dict').toPandas()
section_dict['code'] = section_dict.index

In [141]:

train_sample_query = '''
select label,first_day,max_score,feat_1,feat_cnt,feat_max_score,feat_avg_score,feat_avg_tic from user_kposminin.la_yaca_ccall_20161103 
where substr(md5(phone_num),1,1) = '1' or label = 1
'''

train = hc.sql(train_sample_query) \
    .toPandas()

In [132]:
test_sample = hc.sql('''select label,first_day,max_score,feat_1,feat_cnt,feat_max_score,feat_avg_score,feat_avg_tic 
   from user_kposminin.la_yaca_ccall_20161108 where substr(md5(phone_num),1,1) = "#k"'''.replace('#k','5')) \
    .toPandas()

In [174]:
import sklearn.metrics as m
auc_maxs = m.roc_auc_score(y_true = test_sample['label'], y_score = test_sample['max_score'])
print('Test max score ROC AUC {}.'.format(auc_maxs))

Test max score ROC AUC 0.711952796491.


In [172]:
#sd = section_dict[['section_ind','code']].values.tolist()
btc = max([e[1] for e in sd]) + 5
with open('/data1/share/kosm/data/train_feat_full.txt','w') as f:
    for vr in train[train['feat_cnt'] != ''][['label','max_score','feat_1','feat_cnt','feat_max_score','feat_avg_score','feat_avg_tic']].values:
        for r in sd:
            for i in range(5):
                vr[i+2] = vr[i+2].replace(r[0],str(r[1]+ btc * i + 1))
        f.write('{} 0:{} {} {} {} {} {}\n'.format(vr[0],vr[1],vr[2],vr[3],vr[4],vr[5],vr[6]))

with open('/data1/share/kosm/data/test_feat_full.txt','w') as f:
    for vr in train[train['feat_cnt'] != ''][['label','max_score','feat_1','feat_cnt','feat_max_score','feat_avg_score','feat_avg_tic']].values:
        for r in sd:
            for i in range(5):
                vr[i+2] = vr[i+2].replace(r[0],str(r[1]+450*i + 1))
        f.write('{} 0:{} {} {} {} {} {}\n'.format(vr[0],vr[1],vr[2],vr[3],vr[4],vr[5],vr[6]))


In [223]:
#! head -5 /data1/share/kosm/data/test_1.txt
! /data1/share/LightGBM/lightgbm  config=/home/k.osminin/lgbm_files/train.conf num_trees=100 num_leaves=63 \
  data=/data1/share/kosm/data/train_feat_full.txt valid=/data1/share/kosm/data/test_feat_full.txt > /home/k.osminin/lgbm_files/yaca_lgbm.log
! cat /home/k.osminin/lgbm_files/train.conf /home/k.osminin/lgbm_files/yaca_lgbm.log > /home/k.osminin/lgbm_files/yaca_feat_full.log
! cat /home/k.osminin/lgbm_files/yaca_feat_full.log

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Warning] Ignoring feature Column_1, only has one value
[LightGBM] [Warning] Ignoring feature Column_2, only has one value
[LightGBM] [Warning] Ignoring feature Column_3, only has one value
[LightGBM] [Warning] Ignoring feature Column_6, only has one value
[LightGBM] [Warning] Ignoring feature Column_7, only has one value
[LightGBM] [Warning] Ignoring feature Column_8, only has one value
[LightGBM] [Warning] Ignoring feature Column_9, only has one value
[LightGBM] [Warning] Ignoring feature Column_10, only has one value
[LightGBM] [Warning] Ignoring feature Column_15, only has one value
[LightGBM] [Warning] Ignoring feature Column_16, only has one value
[LightGBM] [Warning] Ignoring feature Column_20, only has one value
[LightGBM] [Warning] Ignoring feature Column_22, only has one value
[LightGBM] [Warning] Ignoring feature Column_24, only has one value
[LightGBM] [Warning] Ignoring feature Column_26, only has one value
[LightGBM

In [181]:
#Write full test file and rename columns to make it dense
with open('/data1/share/kosm/data/full_many_feat_test.txt','w') as f:
    for k in list('0123456789abcdef'):
        batch = hc.sql('''select label,max_score,feat_1,feat_cnt,feat_max_score,feat_avg_score,feat_avg_tic 
   from user_kposminin.la_yaca_ccall_20161108 where substr(md5(phone_num),1,1) = "#k"'''.replace('#k',k)) \
        .collect()
        for vr in batch:
            vr = list(vr)
            if len(vr[2])>2:
                for r in sd:
                    for i in range(5):
                        vr[i+2] = vr[i+2].replace(r[0],str(r[1]+ btc * i + 1))
                f.write('{} 0:{} {} {} {} {} {}\n'.format(vr[0],vr[1],vr[2],vr[3],vr[4],vr[5],vr[6]))
    


In [224]:
#len(sd)
! /data1/share/LightGBM/lightgbm  config=/home/k.osminin/lgbm_files/test.conf input_model=lgbm.model \
   data=/data1/share/kosm/data/full_many_feat_test.txt output_result=/data1/share/kosm/data/full_test_predict.txt
#! perl -ne 'print substr($_,0,1)' /data1/share/kosm/data/full_test.txt > /data1/share/kosm/data/full_test_label.txt
#os.popen('''perl -ne '@a=split(" ",$_);print @a[0] . " " . substr(@a[1],2) . "\n" ' /data1/share/kosm/data/full_many_feat_test.txt > /data1/share/kosm/data/full_many_feat_test_label_maxsc.txt''').read()


[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading 65 models
[LightGBM] [Info] Finished initializing prediction
[LightGBM] [Info] Finished prediction


In [225]:
scores = [float(e.strip()) for e in open('/data1/share/kosm/data/full_test_predict.txt','r')]
#labels_maxsc = [[int(e.split(' ')[0]),float(e.split(' ')[1].strip())] for e in open('/data1/share/kosm/data/full_many_feat_test_label_maxsc.txt','r')]
#labels = [e[0] for e in labels_maxsc]
#maxsc = [e[1] for e in labels_maxsc]
len(scores),len(labels),len(maxsc)


(18841723, 18841723, 18841723)

In [227]:

import sklearn.metrics as m
auc_many_feat = m.roc_auc_score(y_true = labels, y_score = scores)
auc_maxsc = m.roc_auc_score(y_true = labels, y_score = maxsc)
print('Full test Many feat ROC AUC {}.'.format(auc_many_feat))
print('Full test Max score ROC AUC {}.'.format(auc_maxsc))

Full test Many feat ROC AUC 0.697133411849.
Full test Max score ROC AUC 0.693977861324.


In [229]:
data = sorted(zip(scores,labels),reverse = True)
def nvl(a,b):
    if a:
        return a
    else:
        return b
    
print('\nLift table for LGBM yaca\n\n{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}'.format(
        'q','pos_cnt','pos%','cnt','pos/cnt','lift','score'))
pos_tot = sum([e[1] for e in data])
pos_share_tot = float(pos_tot) / len(data)
for q in [0.9999,0.9998,0.9997,0.9995,0.999,0.998,0.997,0.995,0.99,0.98,0.95,0.9,0.8,0.7,0.5,0.3,0]:
    pos_cnt = sum([e[1] for e in data[:int((1-q)*len(data))]])
    cnt = int((1-q)*len(data))
    pos_share = float(pos_cnt)/cnt
    print('{:<10}\t{:<10}\t{:<10.2%}\t{:<10}\t{:<10.5%}\t{:<10.2f}\t{:<10.5f}'.format(q,
                                                                                      pos_cnt,
                                                                                      float(pos_cnt)/pos_tot,
                                                                                      cnt,pos_share,
                                                                                      pos_share/pos_share_tot,
                                                                                      nvl(data[int((1-q)*len(data))-1][0],-20)
                                                                                     ))


Lift table for LGBM yaca

q         	pos_cnt   	pos%      	cnt       	pos/cnt   	lift      	score     
0.9999    	31        	0.45%     	1884      	1.64544%  	44.70     	0.95867   
0.9998    	70        	1.01%     	3768      	1.85775%  	50.47     	0.95443   
0.9997    	130       	1.87%     	5652      	2.30007%  	62.49     	0.95174   
0.9995    	173       	2.49%     	9420      	1.83652%  	49.90     	0.94957   
0.999     	344       	4.96%     	18841     	1.82581%  	49.61     	0.93615   
0.998     	563       	8.12%     	37683     	1.49404%  	40.59     	0.90853   
0.997     	721       	10.40%    	56525     	1.27554%  	34.66     	0.87321   
0.995     	798       	11.51%    	94208     	0.84706%  	23.01     	0.81508   
0.99      	952       	13.73%    	188417    	0.50526%  	13.73     	0.75422   
0.98      	1239      	17.87%    	376834    	0.32879%  	8.93      	0.70997   
0.95      	1724      	24.86%    	942086    	0.18300%  	4.97      	0.61780   
0.9       	2382      	34.35%    	1884172   	0.126

In [231]:
data = sorted(zip(maxsc,labels),reverse = True)
def nvl(a,b):
    if a:
        return a
    else:
        return b
    
print('\nLift table for Max score\n\n{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}'.format(
        'q','pos_cnt','pos%','cnt','pos/cnt','lift','score'))
pos_tot = sum([e[1] for e in data])
pos_share_tot = float(pos_tot) / len(data)
for q in [0.9999,0.9998,0.9997,0.9995,0.999,0.998,0.997,0.995,0.99,0.98,0.95,0.9,0.8,0.7,0.5,0.3,0]:
    pos_cnt = sum([e[1] for e in data[:int((1-q)*len(data))]])
    cnt = int((1-q)*len(data))
    pos_share = float(pos_cnt)/cnt
    print('{:<10}\t{:<10}\t{:<10.2%}\t{:<10}\t{:<10.5%}\t{:<10.2f}\t{:<10.5f}'.format(q,
                                                                                      pos_cnt,
                                                                                      float(pos_cnt)/pos_tot,
                                                                                      cnt,pos_share,
                                                                                      pos_share/pos_share_tot,
                                                                                      nvl(data[int((1-q)*len(data))-1][0],-20)
                                                                                     ))


Lift table for Max score

q         	pos_cnt   	pos%      	cnt       	pos/cnt   	lift      	score     
0.9999    	89        	1.28%     	1884      	4.72399%  	128.35    	-2.72177  
0.9998    	114       	1.64%     	3768      	3.02548%  	82.20     	-3.08295  
0.9997    	195       	2.81%     	5652      	3.45011%  	93.74     	-3.19479  
0.9995    	280       	4.04%     	9420      	2.97240%  	80.76     	-3.37581  
0.999     	447       	6.45%     	18841     	2.37249%  	64.46     	-3.81621  
0.998     	594       	8.57%     	37683     	1.57631%  	42.83     	-4.28751  
0.997     	723       	10.43%    	56525     	1.27908%  	34.75     	-4.76759  
0.995     	793       	11.43%    	94208     	0.84175%  	22.87     	-5.80580  
0.99      	898       	12.95%    	188417    	0.47660%  	12.95     	-6.75822  
0.98      	1104      	15.92%    	376834    	0.29297%  	7.96      	-7.97809  
0.95      	1767      	25.48%    	942086    	0.18756%  	5.10      	-8.39899  
0.9       	2337      	33.70%    	1884172   	0.124